## Database Engineering

In [160]:
#Dependencies
import pandas as pd
import csv

In [161]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func
# Import modules to declare columns and column data types
from sqlalchemy import Column, Integer, String, Float
# Import and establish Base for which classes will be constructed 
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [162]:
#Read the csv files into pandas dataframes
migrant_df=pd.read_csv('MissingMigrants_Global_2019 2.csv')

In [163]:
migrant_df.head()

,Web_ID,Region_of_Incident,Reported_Date,Reported_Year,Month,Number_Dead,Minimum_Estimated_Number_of_Missing,Total_Dead_and_Missing,Number_of_Survivors,Number_of_Females,...,Number_of_Children,Cause_of_Death,Location_Description,Information_Source,Lat,Lon,Migration_Route,URL,UNSD_Geographical_Grouping,Source_Quality
0,51591,Mediterranean,"March 28, 2019",2019,Mar,0,2,2,36,0,...,0,Presumed drowning,"Off the coast of Chios, Greece",Hellenic Coast Guard via IOM Greece,38.362369,26.172509,Eastern Mediterranean,Not Available,Uncategorized,5
1,51588,Mediterranean,"March 26, 2019",2019,Mar,4,0,4,11,3,...,1,Presumed drowning,"Off the coast of Ayvacık district, Çanakkale p...",Turkish Coast Guard via IOM Turkey,39.441976,26.378816,Eastern Mediterranean,http://bit.ly/2YmiPAN,Uncategorized,5
2,51589,Mediterranean,"March 26, 2019",2019,Mar,1,0,1,0,0,...,0,Presumed drowning,"Body recovered on Playa del Tarajal, Ceuta, Sp...","Ceuta al día, El Pueblo de Ceuta",35.871902,-5.343038,Western Mediterranean,"http://bit.ly/2uyj7qO, http://bit.ly/2uwj5zC",Uncategorized,3
3,51590,Mediterranean,"March 26, 2019",2019,Mar,1,0,1,0,0,...,0,Presumed drowning,"Body recovered on beach near Tetouan, Morocco ...",El Pueblo de Ceuta,35.635116,-5.275650,Western Mediterranean,http://bit.ly/2uwj5zC,Uncategorized,1
4,51587,Central America,"March 25, 2019",2019,Mar,1,0,1,0,0,...,0,Fall from train,"Train tracks in Teacalco, Tlaxcala, Mexico","Megalópolis, Línea de contraste",19.334475,-98.069824,Not Available,"http://bit.ly/2uvDIvH, http://bit.ly/2TXAFLS",Central America,3


In [164]:
#Create and connect to the sqlite database
engine = create_engine("sqlite:///migrant.sqlite", echo=False)
conn = engine.connect()

In [165]:
# Create Migrant CLasses
# ----------------------------------
class Migrant(Base):
    __tablename__ = 'migrant'
    id = Column(Integer, primary_key=True)
    web_id = Column(String(255))
    region_of_incident = Column(String(255))
    reported_date = Column(String(255))
    reported_year = Column(String(255))
    month = Column(String(255))
    number_dead = Column(Integer)
    minimum_estimated_number_of_missing = Column(Integer)
    total_dead_and_missing = Column(Integer)
    number_of_survivors = Column(Integer)
    number_of_females = Column(Integer)
    number_of_males = Column(Integer)
    number_of_children = Column(Integer)
    cause_of_death = Column(String(255))
    location_description = Column(String(255))
    information_source = Column(String(255))
    Lat = Column(Float(255))
    Lon = Column(Float(255))
    migration_route = Column(String(255))
    URL = Column(String(255))
    unsd_geographical_grouping = Column(String(255))
    source_quality = Column(Integer)
    

In [166]:
# Create both the table calsses within the database
Base.metadata.create_all(conn)

In [167]:
# Adding migrant data
migrant_df.to_sql(name ='migrant', con=conn, if_exists='append', index=False)

In [168]:
session = Session(bind=engine)

In [169]:
# Use the Inspector to explore the database and print the table names
### BEGIN SOLUTION
inspector = inspect(engine)
inspector.get_table_names()
### END SOLUTION

['migrant']

In [170]:
# Use `engine.execute` to select and display the first 10 rows from the station table
### BEGIN SOLUTION
engine.execute('SELECT * FROM migrant LIMIT 10').fetchall()
### END SOLUTION

[(1, '51591', 'Mediterranean', 'March 28, 2019', '2019', 'Mar', 0, 2, 2, 36, 0, 2, 0, 'Presumed drowning', 'Off the coast of Chios, Greece', 'Hellenic Coast Guard via IOM Greece', 38.362368700000005, 26.17250947, 'Eastern Mediterranean', 'Not Available', 'Uncategorized', 5),
 (2, '51588', 'Mediterranean', 'March 26, 2019', '2019', 'Mar', 4, 0, 4, 11, 3, 0, 1, 'Presumed drowning', 'Off the coast of Ayvacık district, Çanakkale province, Turkey', 'Turkish Coast Guard via IOM Turkey', 39.44197559, 26.378816200000003, 'Eastern Mediterranean', 'http://bit.ly/2YmiPAN', 'Uncategorized', 5),
 (3, '51589', 'Mediterranean', 'March 26, 2019', '2019', 'Mar', 1, 0, 1, 0, 0, 0, 0, 'Presumed drowning', 'Body recovered on Playa del Tarajal, Ceuta, Spain (Strait of Gibraltar)', 'Ceuta al día, El Pueblo de Ceuta', 35.87190188, -5.343037666, 'Western Mediterranean', 'http://bit.ly/2uyj7qO, http://bit.ly/2uwj5zC', 'Uncategorized', 3),
 (4, '51590', 'Mediterranean', 'March 26, 2019', '2019', 'Mar', 1, 0, 1,